In [1]:

#comment out for local testing out of Main Sequence Platform
import dotenv
dotenv.load_dotenv('../.env.dev_local')

from mainsequence.virtualfundbuilder.portfolio_interface import PortfolioInterface
from mainsequence.virtualfundbuilder.agent_interface import TDAGAgent

2025-03-06T08:55:30.389436Z [debug    ] Getting Auth Headers TS_ORM    application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:138 in refresh_headers())
2025-03-06T08:55:30.398070Z [debug    ] Auth Headers Acquired          application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:145 in refresh_headers())
2025-03-06T08:55:30.438041Z [debug    ] Requesting GET from http://192.168.178.17:8000/pods/api/projects/get_user_default_project/ application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:187 in make_request())
2025-03-06T08:55:30.613679Z [debug    ] Getting Auth Headers TS_ORM    application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:138 in refresh_headers())
2025-03-06T08:55:30.615805Z [debug    ] Auth Headers Acquired          application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:145 in refresh_headers())
2025-03-06T08:55:30.617661Z [debu

# Connect to the TDAG-LLM Agent

In [2]:
tdag_agent = TDAGAgent()

2025-03-06T08:55:32.683559Z [debug    ] Send default documentation to Backend application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at __main__.py:192 in send_default_configuration())
2025-03-06T08:55:32.685668Z [debug    ] Requesting POST from http://192.168.178.17:8000/tdag-gpt/register_default_configuration/ application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:187 in make_request())
2025-03-06T08:55:32.803835Z [info     ] Setup TDAG Agent successfull   application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at agent_interface.py:28 in __init__())


# Create a custom Strategy

# Get the configuration for the full portfolio based on our strategy

In [3]:
PortfolioInterface.list_configurations()

[]

In [4]:
from mainsequence.virtualfundbuilder.contrib.signals import MarketCap

portfolio = tdag_agent.generate_portfolio(MarketCap, signal_description="Create me a market cap portfolio using AAPL and GOOG")

portfolio

2025-03-06T08:55:41.793356Z [debug    ] Register signal: MarketCap     application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:678 in _send_strategy_to_registry())
2025-03-06T08:55:41.795650Z [debug    ] Requesting POST from http://192.168.178.17:8000/tdag-gpt/register_strategy/ application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:187 in make_request())
2025-03-06T08:55:41.938279Z [info     ] Get configuration for MarketCap ... application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at agent_interface.py:43 in generate_portfolio())
2025-03-06T08:55:41.940540Z [debug    ] Requesting POST from http://192.168.178.17:8000/tdag-gpt/create_configuration_for_strategy/ application_name=ms-sdk data_source_id=28 job_run_id=None project_id=43 (at utils.py:187 in make_request())
2025-03-06T08:56:39.270643Z [info     ] Replaced 'None' in configuration with None at root.portfolio_build_configuration.backtesting_weights_con

Configuration Name: -
portfolio_build_configuration:
  assets_configuration:
    asset_universe:
      asset_filters:
      - asset_type: cash_equity
        execution_venue__symbol: alpaca
        symbol: AAPL
      - asset_type: cash_equity
        execution_venue__symbol: alpaca
        symbol: GOOG
    price_type: vwap
    prices_configuration:
      bar_frequency_id: 1d
      intraday_bar_interpolation_rule: ffill
      upsample_frequency_id: 1d
  backtesting_weights_configuration:
    rebalance_strategy_configuration:
      calendar: 24/7
      signal_assets_configuration:
        asset_universe:
          asset_filters:
          - asset_type: cash_equity
            execution_venue__symbol: alpaca
            symbol: AAPL
          - asset_type: cash_equity
            execution_venue__symbol: alpaca
            symbol: GOOG
    rebalance_strategy_name: ImmediateSignal
    signal_weights_configuration:
      num_top_assets: None
      signal_assets_configuration:
        asset_

# Run the strategy

In [ ]:
res = portfolio.run()
res.head()

# Plot the portfolio results

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Create the plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=res.index, 
        y=res["portfolio"], 
        mode="lines+markers",
        name="Portfolio Value",
        line=dict(color='blue', width=1),
        marker=dict(size=2)
    )
)

# Customize layout
fig.update_layout(
    title="Portfolio Value Over Time",
    xaxis_title="Date",
    yaxis_title="Portfolio Value",
    template="plotly_white",
    hovermode="x unified"
)

# Show the plot
fig.show()